<a href="https://colab.research.google.com/github/aswit3/Analystics_Vidhya_Sentiment_Analysis_Medical/blob/master/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
tr = pd.read_csv("drive/My Drive/datasets/Sentiment_Analysis_Analytics_vidhya_Medical/train_F3WbcTw.csv")
te = pd.read_csv("drive/My Drive/datasets/Sentiment_Analysis_Analytics_vidhya_Medical/test_tOlRoBf.csv")

In [80]:
tr.shape, te.shape

((5279, 4), (2924, 3))

In [81]:
tr.head(2)

,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2


In [82]:
te.head(2)

,unique_hash,text,drug
0,9e9a8166b84114aca147bf409f6f956635034c08,"256 (previously stable on natalizumab), with 5...",fingolimod
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,On fingolimod and have been since December 201...,fingolimod


In [0]:
del tr["unique_hash"]
del te["unique_hash"]

In [84]:
len(tr["drug"].value_counts()), len(te["drug"].value_counts())

(102, 95)

In [85]:
tr.shape, te.shape

((5279, 3), (2924, 2))

In [86]:
tr_sentiment = tr["sentiment"]
tr_sentiment.shape

(5279,)

In [0]:
del tr["sentiment"]

In [88]:
tr.shape, te.shape

((5279, 2), (2924, 2))

In [0]:
trte = tr.append(te, ignore_index=True)

In [90]:
trte.shape

(8203, 2)

In [91]:
len(trte["drug"].value_counts())

111

In [0]:
import re

clean_list = []

for i, unclean_text in enumerate(trte["text"]):
  clean_text = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', '', unclean_text)
  clean_list.append(clean_text)

In [0]:
trte["text"] = clean_list

In [0]:
import re, string, unicodedata
import inflect
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [95]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

In [0]:
words_ls = []
for i, unclean_text in enumerate(trte["text"]):
  words = nltk.word_tokenize(unclean_text)
  words = normalize(words)
  words_ls.append(words)

In [0]:
trte['text'] = words_ls

In [0]:
text_tokens =trte['text']

In [0]:
import gensim

In [44]:
model_w2v = gensim.models.Word2Vec(
            text_tokens,
            size=200, # desired no. of features/independent variables 
            window=5, # context window size
            min_count=2,
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 2, # no.of cores
            seed = 34)

model_w2v.train(text_tokens, total_examples= len(tr['text']), epochs=10)

(16454584, 17206770)

In [45]:
model_w2v.save("drive/My Drive/datasets/Sentiment_Analysis_Analytics_vidhya_Medical/word2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [100]:
from gensim.models import Word2Vec
model_w2v = Word2Vec.load("drive/My Drive/datasets/Sentiment_Analysis_Analytics_vidhya_Medical/word2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [47]:
model_w2v.wv.most_similar(positive="fingolimod")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('rebound', 0.6422704458236694),
 ('gilenya', 0.6394204497337341),
 ('natalizumab', 0.6385238170623779),
 ('teriflunomide', 0.6147090196609497),
 ('n257', 0.6009764671325684),
 ('alemtuzumab', 0.6003761887550354),
 ('categorizes', 0.5999492406845093),
 ('dimethyl', 0.597201943397522),
 ('one thousand, one hundred and sixty', 0.5943523645401001),
 ('lemtradatm', 0.585485577583313)]

In [0]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary
                         
            continue
    if count != 0:
        vec /= count
    return vec

In [102]:
import numpy as np
wordvec_arrays = np.zeros((len(text_tokens), 200))

for i in range(len(text_tokens)):
    wordvec_arrays[i,:] = word_vector(text_tokens[i], 200)
    
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(8203, 200)

In [0]:
tr_sentiment = tr_sentiment

In [104]:
from sklearn.model_selection import train_test_split

train_w2v = wordvec_df.iloc[:5279,:]
test_w2v = wordvec_df.iloc[5279:,:]


_, _, ytrain, yvalid = train_test_split(train_w2v, tr_sentiment,  
                                                          random_state=42, 
                                                          test_size=0.3)



print(train_w2v.shape, tr_sentiment.shape)

xtrain_w2v = train_w2v.iloc[ytrain.index,:]
xvalid_w2v = train_w2v.iloc[yvalid.index,:]

(5279, 200) (5279,)


In [0]:
import tensorflow as tf
import keras.layers as layers
from keras.models import Model
from keras.datasets import imdb
from gensim.models import Word2Vec
from gensim.models import FastText
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Input,Embedding,Dense,Flatten
from sklearn.metrics import accuracy_score,classification_report
from sklearn.metrics import f1_score

In [106]:
epochs = 25
batch_size = 1024
loss = "sparse_categorical_crossentropy"
optimizer = "adam"
metrics = ["accuracy"]

from keras import models

callbacks = [EarlyStopping(monitor='val_loss', patience=2),
            ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

# Build neural network
model = models.Sequential()
model.add(Dense(512, activation='relu', input_shape=(200,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss=loss,optimizer=optimizer,metrics= metrics)
model.fit(xtrain_w2v,ytrain,epochs=epochs,batch_size=batch_size,callbacks=callbacks,validation_data=(xvalid_w2v,yvalid))

Train on 3695 samples, validate on 1584 samples
Epoch 1/25
3695/3695 [==============================] - 1s 204us/step - loss: 0.9446 - acc: 0.5778 - val_loss: 0.7940 - val_acc: 0.7210
Epoch 2/25
3695/3695 [==============================] - 0s 49us/step - loss: 0.7965 - acc: 0.7261 - val_loss: 0.7737 - val_acc: 0.7210
Epoch 3/25
3695/3695 [==============================] - 0s 43us/step - loss: 0.7611 - acc: 0.7261 - val_loss: 0.7662 - val_acc: 0.7210
Epoch 4/25
3695/3695 [==============================] - 0s 57us/step - loss: 0.7544 - acc: 0.7261 - val_loss: 0.7520 - val_acc: 0.7210
Epoch 5/25
3695/3695 [==============================] - 0s 60us/step - loss: 0.7388 - acc: 0.7261 - val_loss: 0.7443 - val_acc: 0.7210
Epoch 6/25
3695/3695 [==============================] - 0s 58us/step - loss: 0.7289 - acc: 0.7261 - val_loss: 0.7321 - val_acc: 0.7210
Epoch 7/25
3695/3695 [==============================] - 0s 59us/step - loss: 0.7194 - acc: 0.7261 - val_loss: 0.7330 - val_acc: 0.7210
Epoch 

In [108]:
predictions = model.predict(xvalid_w2v)
predictions = predictions.argmax(axis = -1)
df = pd.DataFrame()
df['new_col'] = predictions
f1_score(yvalid, df['new_col'], average='micro')

0.7316919191919192

In [0]:
test_pred = model.predict(test_w2v)
test_pred = test_pred.argmax(axis = -1)

te_res = pd.read_csv("drive/My Drive/datasets/Sentiment_Analysis_Analytics_vidhya_Medical/test_tOlRoBf.csv")
te_res['sentiment'] = test_pred

submission = te_res[['unique_hash','sentiment']]
submission.to_csv('model_result.csv', index=False)